In [1]:
import nflreadpy as nfl
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set display options
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_width_chars(200)

print("Imports successful")

Imports successful


In [2]:
charting = nfl.load_participation()

In [3]:
charting

nflverse_game_id,old_game_id,play_id,possession_team,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type,offense_names,defense_names,offense_positions,defense_positions,offense_numbers,defense_numbers
str,str,f64,str,str,str,i32,str,i32,str,str,str,i32,i32,f64,f64,bool,str,str,str,str,str,str,str,str,str
"""2024_01_TEN_CHI""","""2024090802""",40.0,"""TEN""",null,"""1 FS, 2 LB, 1 OLB, 2 RB, 3 TE,…",0,"""4 CB, 1 FS, 2 ILB, 1 K, 1 OLB,…",0,"""00-0037106;00-0037129;00-00390…","""00-0037493;00-0039430;00-00394…","""00-0037106;00-0037129;00-00390…",11,11,null,null,false,"""""","""""",null,"""Mike Brown;James Williams;Jayl…","""Jaylon Jones;Josh Blackwell;Ty…","""FS;LB;LB;OLB;RB;RB;TE;TE;TE;WR…","""CB;CB;CB;CB;FS;ILB;ILB;K;OLB;R…","""44;52;92;57;36;2;84;85;81;15;1…","""21;39;29;32;22;45;57;8;92;20;1…"
"""2024_01_TEN_CHI""","""2024090802""",63.0,"""TEN""","""UNDER CENTER""","""1 C, 1 QB, 1 RB, 4 T, 1 TE, 3 …",7,"""3 CB, 2 DE, 2 DT, 1 FS, 2 ILB,…",0,"""00-0036391;00-0036292;00-00381…","""00-0036391;00-0039152;00-00352…","""00-0036292;00-0038111;00-00383…",11,11,null,null,false,"""""","""""",null,"""Lloyd Cushenberry III;Will Lev…","""Jaylon Johnson;Kyler Gordon;Te…","""C;QB;RB;T;T;T;T;TE;WR;WR;WR""","""CB;CB;CB;DE;DE;DT;DT;FS;ILB;IL…","""79;8;20;75;78;77;55;85;10;83;0""","""1;6;32;95;98;97;99;31;49;53;9"""
"""2024_01_TEN_CHI""","""2024090802""",85.0,"""TEN""","""SHOTGUN""","""1 C, 1 QB, 1 RB, 4 T, 1 TE, 3 …",7,"""3 CB, 2 DE, 2 DT, 1 FS, 2 ILB,…",4,"""00-0036391;00-0036292;00-00381…","""00-0036391;00-0039152;00-00352…","""00-0036292;00-0038111;00-00390…",11,11,null,2.55,true,"""SWING""","""ZONE_COVERAGE""","""COVER_2""","""Lloyd Cushenberry III;Will Lev…","""Jaylon Johnson;Kyler Gordon;Ty…","""C;QB;RB;T;T;T;T;TE;WR;WR;WR""","""CB;CB;CB;DE;DE;DT;DT;FS;ILB;IL…","""79;8;20;75;78;77;55;85;83;0;16""","""1;6;29;95;98;97;99;31;49;53;9"""
"""2024_01_TEN_CHI""","""2024090802""",108.0,"""TEN""","""SHOTGUN""","""1 C, 1 QB, 1 RB, 4 T, 1 TE, 3 …",5,"""3 CB, 2 DE, 1 DT, 1 FS, 2 ILB,…",4,"""00-0036391;00-0036292;00-00381…","""00-0036391;00-0039152;00-00390…","""00-0036292;00-0038111;00-00390…",11,11,null,2.96,false,"""DEEP OUT""","""MAN_COVERAGE""","""2_MAN""","""Lloyd Cushenberry III;Will Lev…","""Jaylon Johnson;Kyler Gordon;Ty…","""C;QB;RB;T;T;T;T;TE;WR;WR;WR""","""CB;CB;CB;DE;DE;DT;FS;ILB;ILB;O…","""79;8;2;75;78;77;55;85;83;0;16""","""1;6;29;95;98;99;31;49;53;52;9"""
"""2024_01_TEN_CHI""","""2024090802""",131.0,"""TEN""",null,"""1 DB, 1 FS, 1 ILB, 2 LB, 1 LS,…",0,"""2 CB, 1 FB, 1 FS, 2 ILB, 1 OLB…",0,"""00-0037106;00-0037129;00-00393…","""00-0039372;00-0037493;00-00375…","""00-0037106;00-0037129;00-00350…",11,11,null,null,false,"""""","""""",null,"""Jarvis Brownlee Jr.;Mike Brown…","""Jaylon Jones;Josh Blackwell;Kh…","""DB;FS;ILB;LB;LB;LS;OLB;P;RB;TE…","""CB;CB;FB;FS;ILB;ILB;OLB;RB;SS;…","""29;44;50;52;92;46;57;4;36;85;1…","""21;39;35;22;45;57;92;20;36;11;…"
"""2024_01_TEN_CHI""","""2024090802""",162.0,"""CHI""","""UNDER CENTER""","""1 C, 1 G, 1 QB, 1 RB, 3 T, 2 T…",7,"""2 CB, 2 DT, 2 FS, 2 ILB, 1 NT,…",0,"""00-0034114;00-0033548;00-00363…","""00-0034114;00-0035649;00-00399…","""00-0033548;00-0036374;00-00348…",11,11,null,null,false,"""""","""""",null,"""Coleman Shelton;Nate Davis;Cal…","""Chidobe Awuzie;L'Jarius Sneed;…","""C;G;QB;RB;T;T;T;TE;TE;WR;WR""","""CB;CB;DT;DT;FS;FS;ILB;ILB;NT;O…","""65;64;18;4;76;70;58;14;85;13;2""","""13;38;69;98;28;37;56;50;93;58;…"
"""2024_01_TEN_CHI""","""2024090802""",184.0,"""CHI""","""SHOTGUN""","""1 C, 1 G, 1 QB, 1 RB, 3 T, 1 T…",5,"""3 CB, 1 DT, 2 FS, 2 ILB, 1 NT,…",4,"""00-0034114;00-0033548;00-00363…","""00-0034114;00-0035649;00-00399…","""00-0033548;00-0036374;00-00381…",11,11,null,1.1,false,"""SCREEN""","""ZONE_COVERAGE""","""COVER_2""","""Coleman Shelton;Nate Davis;Cal…","""Chidobe Awuzie;L'Jarius Sneed;…","""

In [9]:
pbp = nfl.load_pbp(seasons=2025)

print("PLAY-BY-PLAY DATA STRUCTURE")
print("=" * 80)
print(f"Shape: {pbp.shape}")
print(f"\nColumns ({len(pbp.columns)}):")
for i, col in enumerate(pbp.columns, 1):
    print(f"{i:3}. {col}")

PLAY-BY-PLAY DATA STRUCTURE
Shape: (22981, 372)

Columns (372):
  1. play_id
  2. game_id
  3. old_game_id
  4. home_team
  5. away_team
  6. season_type
  7. week
  8. posteam
  9. posteam_type
 10. defteam
 11. side_of_field
 12. yardline_100
 13. game_date
 14. quarter_seconds_remaining
 15. half_seconds_remaining
 16. game_seconds_remaining
 17. game_half
 18. quarter_end
 19. drive
 20. sp
 21. qtr
 22. down
 23. goal_to_go
 24. time
 25. yrdln
 26. ydstogo
 27. ydsnet
 28. desc
 29. play_type
 30. yards_gained
 31. shotgun
 32. no_huddle
 33. qb_dropback
 34. qb_kneel
 35. qb_spike
 36. qb_scramble
 37. pass_length
 38. pass_location
 39. air_yards
 40. yards_after_catch
 41. run_location
 42. run_gap
 43. field_goal_result
 44. kick_distance
 45. extra_point_result
 46. two_point_conv_result
 47. home_timeouts_remaining
 48. away_timeouts_remaining
 49. timeout
 50. timeout_team
 51. td_team
 52. td_player_name
 53. td_player_id
 54. posteam_timeouts_remaining
 55. defteam_timeo

In [10]:
# Look at the most important columns for analysis
key_cols = ['game_id', 'play_id', 'posteam', 'defteam', 'down', 'ydstogo', 
            'yardline_100', 'play_type', 'yards_gained', 'touchdown', 'epa', 'wpa']

# Filter to columns that actually exist
existing_cols = [col for col in key_cols if col in pbp.columns]
print(f"Key columns found: {existing_cols}")
print("\nSample data:")
pbp.select(existing_cols).head(10)

Key columns found: ['game_id', 'play_id', 'posteam', 'defteam', 'down', 'ydstogo', 'yardline_100', 'play_type', 'yards_gained', 'touchdown', 'epa', 'wpa']

Sample data:


game_id,play_id,posteam,defteam,down,ydstogo,yardline_100,play_type,yards_gained,touchdown,epa,wpa
str,f64,str,str,f64,f64,f64,str,f64,f64,f64,f64
"""2025_01_ARI_NO""",1.0,null,null,null,0.0,null,null,null,null,-0.0,-0.0
"""2025_01_ARI_NO""",40.0,"""ARI""","""NO""",null,0.0,35.0,"""kickoff""",0.0,0.0,-0.3527,0.000701
"""2025_01_ARI_NO""",63.0,"""ARI""","""NO""",1.0,10.0,78.0,"""run""",3.0,0.0,-0.190052,-0.014005
"""2025_01_ARI_NO""",85.0,"""ARI""","""NO""",2.0,7.0,75.0,"""pass""",11.0,0.0,1.31734,0.034995
"""2025_01_ARI_NO""",115.0,"""ARI""","""NO""",1.0,10.0,64.0,"""pass""",-11.0,0.0,-1.69436,-0.050684
"""2025_01_ARI_NO""",135.0,"""ARI""","""NO""",2.0,21.0,75.0,"""run""",-2.0,0.0,-1.28415,-0.024458
"""2025_01_ARI_NO""",166.0,"""ARI""","""NO""",3.0,23.0,77.0,"""run""",1.0,0.0,-0.840574,-0.036715
"""2025_01_ARI_NO""",188.0,"""ARI""","""NO""",4.0,22.0,76.0,"""punt""",0.0,0.0,0.650589,0.043584
"""2025_01_ARI_NO""",214.0,"""NO""","""ARI""",1.0,10.0,77.0,"""run""",3.0,0.0,-0.194728,-0.011112


In [11]:
# See what data types we have
pbp.schema

Schema([('play_id', Float64),
        ('game_id', String),
        ('old_game_id', String),
        ('home_team', String),
        ('away_team', String),
        ('season_type', String),
        ('week', Int32),
        ('posteam', String),
        ('posteam_type', String),
        ('defteam', String),
        ('side_of_field', String),
        ('yardline_100', Float64),
        ('game_date', String),
        ('quarter_seconds_remaining', Float64),
        ('half_seconds_remaining', Float64),
        ('game_seconds_remaining', Float64),
        ('game_half', String),
        ('quarter_end', Float64),
        ('drive', Float64),
        ('sp', Float64),
        ('qtr', Float64),
        ('down', Float64),
        ('goal_to_go', Int32),
        ('time', String),
        ('yrdln', String),
        ('ydstogo', Float64),
        ('ydsnet', Float64),
        ('desc', String),
        ('play_type', String),
        ('yards_gained', Float64),
        ('shotgun', Float64),
        ('no_huddle',

In [13]:
player_stats = nfl.load_player_stats(seasons=2025)

print("PLAYER STATS STRUCTURE")
print("=" * 80)
print(f"Shape: {player_stats.shape}")
print(f"\nAll columns ({len(player_stats.columns)}):")
for i, col in enumerate(player_stats.columns, 1):
    print(f"{i:3}. {col}")

PLAYER STATS STRUCTURE
Shape: (9169, 114)

All columns (114):
  1. player_id
  2. player_name
  3. player_display_name
  4. position
  5. position_group
  6. headshot_url
  7. season
  8. week
  9. season_type
 10. team
 11. opponent_team
 12. completions
 13. attempts
 14. passing_yards
 15. passing_tds
 16. passing_interceptions
 17. sacks_suffered
 18. sack_yards_lost
 19. sack_fumbles
 20. sack_fumbles_lost
 21. passing_air_yards
 22. passing_yards_after_catch
 23. passing_first_downs
 24. passing_epa
 25. passing_cpoe
 26. passing_2pt_conversions
 27. pacr
 28. carries
 29. rushing_yards
 30. rushing_tds
 31. rushing_fumbles
 32. rushing_fumbles_lost
 33. rushing_first_downs
 34. rushing_epa
 35. rushing_2pt_conversions
 36. receptions
 37. targets
 38. receiving_yards
 39. receiving_tds
 40. receiving_fumbles
 41. receiving_fumbles_lost
 42. receiving_air_yards
 43. receiving_yards_after_catch
 44. receiving_first_downs
 45. receiving_epa
 46. receiving_2pt_conversions
 47. racr


In [14]:


# Fantasy football key metrics
fantasy_cols = ['player_name', 'position', 'team', 'week', 
                'passing_yards', 'passing_tds', 
                'rushing_yards', 'rushing_tds',
                'receiving_yards', 'receiving_tds', 'receptions',
                'fantasy_points', 'fantasy_points_ppr']

existing_fantasy = [col for col in fantasy_cols if col in player_stats.columns]
print(f"Fantasy columns found: {existing_fantasy}")
print("\nTop  fantasy scorers (Week 1):")
(player_stats
 .filter(pl.col('week') == 1) if 'week' in player_stats.columns else player_stats
 .select(existing_fantasy)
 .sort('fantasy_points_ppr', descending=True)
 .head(10)
)

Fantasy columns found: ['player_name', 'position', 'team', 'week', 'passing_yards', 'passing_tds', 'rushing_yards', 'rushing_tds', 'receiving_yards', 'receiving_tds', 'receptions', 'fantasy_points', 'fantasy_points_ppr']

Top  fantasy scorers (Week 1):


player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,…,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance,fantasy_points,fantasy_points_ppr
str,str,str,str,str,str,i32,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f64,f64,i32,f64,i32,i32,i32,i32,i32,i32,f64,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,f64,i32,i32,i32,i32,i32,f64,f64
"""00-0023459""","""A.Rodgers""","""Aaron Rodgers""","""QB""","""QB""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""PIT""","""NYJ""",22,30,244,4,0,4,-26,0,0,139,173,14,10.204755,6.350381,0,1.755396,1,-1,0,0,0,0,-1.873902,0,0,0,…,0,0,0,0,0,null,null,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,25.66,25.66
"""00-0023853""","""M.Prater""","""Matt Prater""","""K""","""SPEC""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""BUF""","""BAL""",0,0,0,0,0,0,0,0,0,0,0,0,null,null,0,null,0,0,0,0,0,0,null,0,0,0,…,0,3,3,0,0,43,1.0,0,1,1,1,0,0,0,0,0,0,0,0,"""25;43;32""",null,null,100,0,0,2,2,0,0,1.0,1,1,0,0,32,0.0,0.0
"""00-0025565""","""N.Folk""","""Nick Folk""","""K""","""SPEC""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""NYJ""","""PIT""",0,0,0,0,0,0,0,0,0,0,0,0,null,null,0,null,0,0,0,0,0,0,null,0,0,0,…,0,2,2,0,0,51,1.0,0,0,1,0,1,0,0,0,0,0,0,0,"""35;51""",null,null,86,0,0,2,2,0,0,1.0,0,0,0,0,0,0.0,0.0
"""00-0026158""","""J.Flacco""","""Joe Flacco""","""QB""","""QB""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""CLE""","""CIN""",31,45,290,1,2,2,-12,0,0,273,173,13,-1.842128,2.374263,0,1.062271,2,6,0,0,0,1,0.134383,0,0,0,…,0,0,0,0,0,null,null,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,12.2,12.2
"""00-0026190""","""C.Campbell""","""Calais Campbell""","""DE""","""DL""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""ARI""","""NO""",0,0,0,0,0,0,0,0,0,0,0,0,null,null,0,null,0,0,0,0,0,0,null,0,0,0,…,0,0,0,0,0,null,null,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,0.0,0.0
"""00-0026498""","""M.Stafford""","""Matthew Stafford""","""QB""","""QB""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""LA""","""HOU""",21,29,245,1,0,3,-21,0,0,226,68,13,6.090109,5.381517,0,1.084071,4,-2,0,1,0,1,-6.176067,0,0,0,…,0,0,0,0,0,null,null,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,13.6,13.6
"""00-0026858""","""G.Gano""","""Graham Gano""","""K""","""SPEC""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""NYG""","""WAS""",0,0,0,0,0,0,0,0,0,0,0,0,null,null,0,null,0,0,0,0,0,0,null,0,0,0,…,0,2,2,0,0,55,1.0,0,1,0,0,1,0,0,0,0,0,0,0,"""21;55""",null,null,76,0,0,0,0,0,0,null,0,0,0,0,0,0.0,0.0
"""00-0027114""","""T.Morstead""","""Thomas Morstead""","""P""","""SPEC""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""SF""","""SEA""",0,0,0,0,0,0,0,0,0,0,0,0,null,null,0,null,0,0,0,0,0,0,null,0,0,0,…,0,0,0,0,0,null,null,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,0,null,0,0,0,0,0,0.0,0.0
"""00-0027857""","""T.Williams""","""Trent Williams""","""OT""","""OL""","""https://static.www.nfl.com/ima…",2025,1,"""REG""","""SF""","""SEA""",

In [15]:
null_summary = pbp.null_count().transpose(include_header=True, header_name='column', column_names=['null_count'])
null_summary.sort('null_count', descending=True).head(15)

column,null_count
str,u32
"""lateral_sack_player_id""",22981
"""lateral_sack_player_name""",22981
"""lateral_interception_player_id""",22981
"""lateral_interception_player_na…",22981
"""lateral_punt_returner_player_i…",22981
"""lateral_punt_returner_player_n…",22981
"""lateral_kickoff_returner_playe…",22981
"""lateral_kickoff_returner_playe…",22981
"""tackle_for_loss_2_player_id""",22981


In [ ]:
# What kinds of plays do we have?
if 'play_type' in pbp.columns:
    print("PLAY TYPE DISTRIBUTION")
    print("=" * 80)
    play_dist = (pbp
                 .group_by('play_type')
                 .agg(pl.len().alias('count'))
                 .sort('count', descending=True))
    print(play_dist)

# NULL = usually, it's the game summary - each game in pbp has a row which is meant
# to summarize the game. Play type is usually null here. 
# Question: what is considered no_play? When there's a flag? Delay of game? 

PLAY TYPE DISTRIBUTION
shape: (10, 2)
┌─────────────┬───────┐
│ play_type   ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ pass        ┆ 9371  │
│ run         ┆ 6810  │
│ no_play     ┆ 2341  │
│ kickoff     ┆ 1377  │
│ punt        ┆ 961   │
│ null        ┆ 680   │
│ extra_point ┆ 639   │
│ field_goal  ┆ 541   │
│ qb_kneel    ┆ 218   │
│ qb_spike    ┆ 43    │
└─────────────┴───────┘


In [18]:
# Check if we have situational data for coaching analysis
situational_cols = ['down', 'ydstogo', 'yardline_100', 'qtr', 'score_differential', 
                    'shotgun', 'no_huddle', 'posteam_timeouts_remaining']

print("IN-GAME CONTEXT DATA")
print("=" * 80)
for col in situational_cols:
    if col in pbp.columns:
        null_pct = (pbp[col].null_count() / len(pbp)) * 100
        print(f"{col:30} - {null_pct:.1f}% null")
    else:
        print(f"{col:30} - NOT FOUND")

IN-GAME CONTEXT DATA
down                           - 16.5% null
ydstogo                        - 0.0% null
yardline_100                   - 7.3% null
qtr                            - 0.0% null
score_differential             - 5.6% null
shotgun                        - 0.0% null
no_huddle                      - 0.0% null
posteam_timeouts_remaining     - 5.6% null
